In [11]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
import sys
import os
import os.path as osp
sys.path.insert(1, osp.abspath('../'))

## SequenceGraphDataset example

In [ ]:
from modules.torch_dataset.sequence_graph_dataset import SequenceGraphDataset

dataset = SequenceGraphDataset(sequence_path_prefix='datasets/AIC20', 
                               sequence_names=['S01', 'S03', 'S04'], 
                                annotations_filename='gt.txt')

data = dataset[0]     # Access the first processed graph

In [5]:
import pandas as pd
edge = pd.read_json("datasets/AIC20/logs/S03/sequence_graph_edges.json")
edge.head()

,src_node_id,dst_node_id,src_obj_id,dst_obj_id,edge_labels
0,0,14,252,252,1
1,0,15,252,242,0
2,0,16,252,256,0
3,0,17,252,246,0
4,0,18,252,247,0


In [37]:
edge[(edge.src_obj_id == edge.dst_obj_id)].head()

,src_node_id,dst_node_id,src_obj_id,dst_obj_id,edge_labels
0,0,14,252,252,1
17,0,31,252,252,1
31,0,45,252,252,1
48,0,62,252,252,1
67,1,15,242,242,1


## ObjectGraphDataset example

In [13]:
from modules.torch_dataset.object_graph_dataset import ObjectGraphDataset


In [53]:
from models.reid.resnet import resnet101_ibn_a
model = resnet101_ibn_a(model_path="/Users/hectormejia/Documents/TFM/mcmt_gnn/models/reid/resnet101_ibn_a_2.pth", device="cpu")

Loading pretrained model from /Users/hectormejia/Documents/TFM/mcmt_gnn/models/reid/resnet101_ibn_a_2.pth


In [54]:
import torchvision.transforms.v2 as transforms
augmentation = transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.RandomErasing(scale=(0.02, 0.15)),
        transforms.RandomRotation(degrees=0.15),
        transforms.RandomPerspective(distortion_scale=0.25)
    ])

In [55]:
from modules.torch_dataset.object_graph_dataset import ObjectGraphDataset
from modules.data_processor.annotations_processor import AnnotationsProcessor

data_df = AnnotationsProcessor(sequence_path='/Users/hectormejia/Documents/TFM/mcmt_gnn/datasets/AIC20', annotations_filename='gt.txt').consolidate_annotations(['S02'])
dataset = ObjectGraphDataset(data_df, 
                             sequence_path_prefix="/Users/hectormejia/Documents/TFM/mcmt_gnn/datasets/AIC20", 
                             sequence_names=['S02'], 
                             annotations_filename='gt.txt', 
                             reid_model=model,
                             num_ids_per_graph=-1, 
                             embeddings_per_it=48, 
                             resized_img_shape=(224, 224), 
                             orignal_img_shape=(1920, 1080), 
                             augmentation=None)

In [56]:
sampled_df, remaining_obj_ids = dataset.sample_obj_ids(dataset.all_annotations_df, 
                dataset.remaining_obj_ids,
                dataset.num_ids_per_graph)

ValueError: Negative dimensions are not allowed

In [57]:
graph, node_df, edge_df, sampled_df = dataset[0]

Sampling object ids
Computing embeddings


  0%|          | 0/437 [00:00<?, ?it/s]

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


100%|██████████| 437/437 [47:34<00:00,  6.53s/it]


Generating nodes
Generating edges
Processed edges for camera c006. Shapes:  63075 63075 63075 63075
Processed edges for camera c007. Shapes:  42050 42050 42050 42050
Processed edges for camera c008. Shapes:  21025 21025 21025 21025
Processed edges for camera c009. Shapes:  0 0 0 0
Creating PyG graph


In [58]:
from models.mcmt.rgnn import MOTMPNet
import yaml
import torch 

with open("/Users/hectormejia/Documents/TFM/mcmt_gnn/config/training_rgcnn.yml", "r") as config_file:
    config = yaml.safe_load(config_file)
    gnn_arch = config["gnn_arch"]

model_state = torch.load("/Users/hectormejia/Documents/TFM/mcmt_gnn/results/checkpoints/checkpoint_100.pt")
model = MOTMPNet(gnn_arch)
model.load_state_dict(model_state['model'])


<All keys matched successfully>

In [59]:
from modules.inference.inference_module import InferenceModule, InferenceEngineAbstract
inf_module = InferenceModule(model, sampled_df, "/Users/hectormejia/Documents/TFM/mcmt_gnn/datasets/AIC20")

In [67]:
pred_df = inf_module.predict_tracks((graph, node_df, edge_df))

torch.Size([2, 126150])


/Users/hectormejia/Documents/TFM/mcmt_gnn/modules/inference/postprocessing.py:285: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return id_pred, torch.tensor(whole_edges_prediction)


In [66]:
import pandas as pd
gt_df = pd.read_csv("/Users/hectormejia/Documents/TFM/mcmt_gnn/datasets/AIC20/ground_truth_S02.txt", sep=' ', names=['camera','id', 'frame', 'xmin', 'ymin', 'width', 'height', 'xworld', 'yworld'])
gt_df.head()

,camera,id,frame,xmin,ymin,width,heigt,xworld,yworld
0,6,96,1,539,606,273,277,-1,-1
1,6,97,1,374,373,249,219,-1,-1
2,6,98,1,688,345,183,178,-1,-1
3,6,103,1,577,259,132,113,-1,-1
4,6,104,1,712,160,108,92,-1,-1


In [ ]:
from modules.inference.postprocessing import fix_annotation_frames
pred_df.camera = pred_df.camera.apply(lambda cam: int(cam.replace('c', '')))
pred_df.head()

In [79]:
inf_module.data_df = pred_df

In [78]:
inf_module.evaluate_mtmc(gt_df, 0.7)

KeyError: "['frame', 'id', 'height'] not in index"